In [27]:
import pandas as pd
import json
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentPoolEmbeddings, Sentence
import torch
import re
import numpy as np

In [28]:
emb = WordEmbeddings('pl')

In [29]:
with open("recipes_pretty.json") as file:
    data = json.load(file)

# Wyrazy kluczowe

In [30]:
temp = Sentence('jedzenie jednostka narzędzie')
emb.embed(temp)
jedzenie = temp[0].embedding
jednostka = temp[1].embedding
narzedzie = temp[2].embedding

# Przykładowy składnik

### Przygotowanie

In [31]:
ingredient = Sentence(data[0]["ingredients"][0][0])

In [32]:
emb.embed(ingredient)

[Sentence: "3 łyżeczki soku z limonki 
 " - 6 Tokens]

In [33]:
for i in ingredient:
    print(i, i.embedding[:5])

Token: 1 3 tensor([ 0.2346,  0.2216,  0.0498, -0.7787,  0.3949], device='cuda:0')
Token: 2 łyżeczki tensor([-0.1335,  0.2546,  0.0863, -1.0030,  0.3891], device='cuda:0')
Token: 3 soku tensor([-0.3744, -0.0344, -0.4256, -0.9503,  0.4655], device='cuda:0')
Token: 4 z tensor([-0.1382, -0.0861, -0.0652, -0.1541, -0.0471], device='cuda:0')
Token: 5 limonki tensor([-0.4339,  0.4223, -0.4476, -0.9961,  0.3207], device='cuda:0')
 oken: 6 tensor([0., 0., 0., 0., 0.], device='cuda:0')


### Odległość L2 od słowa "jedzenie"

In [34]:
for i in ingredient:
    print(torch.norm(i.embedding - jedzenie))

tensor(5.8111, device='cuda:0')
tensor(5.0761, device='cuda:0')
tensor(5.6460, device='cuda:0')
tensor(5.0353, device='cuda:0')
tensor(5.4801, device='cuda:0')
tensor(4.6423, device='cuda:0')


###  Podobieństwo cosinusowe od słowa jedzenie

In [35]:
def cos(v,u): return (v @ u)/v.norm()/u.norm()

In [36]:
for i in ingredient:
    if i.embedding.norm() > 0.001:
        print(i)
        print(cos(i.embedding, jedzenie))

Token: 1 3
tensor(0.1270, device='cuda:0')
Token: 2 łyżeczki
tensor(0.4890, device='cuda:0')
Token: 3 soku
tensor(0.4162, device='cuda:0')
Token: 4 z
tensor(0.1661, device='cuda:0')
Token: 5 limonki
tensor(0.3724, device='cuda:0')


###  Podobieństwo cosinusowe od słowa jednostka

In [37]:
for i in ingredient:
    if i.embedding.norm() > 0.001:
        print(i)
        print(cos(i.embedding, jednostka))

Token: 1 3
tensor(0.1691, device='cuda:0')
Token: 2 łyżeczki
tensor(0.0952, device='cuda:0')
Token: 3 soku
tensor(0.1207, device='cuda:0')
Token: 4 z
tensor(0.0962, device='cuda:0')
Token: 5 limonki
tensor(0.1066, device='cuda:0')


### Pod. do narzędzie

In [38]:
for i in ingredient:
    if i.embedding.norm() > 0.001:
        print(i)
        print(cos(i.embedding, narzedzie))

Token: 1 3
tensor(0.1972, device='cuda:0')
Token: 2 łyżeczki
tensor(0.3519, device='cuda:0')
Token: 3 soku
tensor(0.1850, device='cuda:0')
Token: 4 z
tensor(0.0672, device='cuda:0')
Token: 5 limonki
tensor(0.1857, device='cuda:0')


In [39]:
for d in data:
    for i in d['ingredients']:
        print(i[0].split())

['3', 'łyżeczki', 'soku', 'z', 'limonki']
['1/2', 'szklanki', 'napoju', 'Rubicon', 'Deluxe', 'Guawa']
['2', 'łyżki', 'miodu']
['2', 'łyżeczki', 'imbiru']
['300ml', 'jogurtu', 'naturalnego', '2%']
['10', 'owoców', 'lychee']
['2', 'banany']
['200g', 'ananasa']
['1', 'szklanka', 'kruszonego', 'lodu']
['miód']
['1', 'szklanka', 'wody', 'kokosowej']
['1', 'szklanka', 'owoców', 'mango', '(świeżych', 'lub', 'mrożonych)']
['1', 'obrany', 'banan']
['200g', 'jogurtu', 'greckiego']
['1', 'łyżka', 'cukru', 'brązowego']
['2', 'szklanki', 'puree', 'z', 'mango']
['1/2', 'szklanki', 'mleczka', 'kokosowego']
['2', 'łyżeczki', 'brązowego', 'cukru']
['1', 'szklanka', 'wody', 'mineralnej', 'niegazowanej']
['4', 'łyżki', 'stołowe', 'nasion', 'Chia']
['2', 'łyżeczki', 'soku', 'z', 'limonki']
['100ml', 'wody', 'kokosowej']
['100g', 'jogurtu', 'naturalnego']
['1/2', 'mango']
['150g', 'truskawek']
['lód']
['Napój', 'Mango', 'Rubicon', 'Deluxe']
['1/2', 'cytryny']
['6', 'mandarynek']
['125g', 'ananasa']
['1', '

['1', 'drobno', 'posiekana', 'papryczka', 'chili', '(opcjonalnie)']
[]
['Składniki', 'na', 'sos:', '•', '2', 'łyżki', 'stołowe', 'sosu', 'ostrygowego']
['2', 'łyżki', 'stołowe', 'sosu', 'po', 'pekińsku']
['2', 'łyżki', 'stołowe', 'wina', 'ryżowego']
['3', 'łyżki', 'stołowe', 'wody']
['350', 'g', 'udek', 'z', 'kurczaka']
['200', 'g', 'świeżego', 'makaronu', 'jajecznego']
['1/2', 'ogórka', 'pokrojonego', 'w', 'cienkie', 'paski']
['1', 'duża', 'marchew,', 'obrana', 'i', 'pokrojona', 'w', 'cienkie', 'paski']
[]
['•', '2', 'łyżki', 'stołowe', 'sosu', 'ostrygowego', 'Lee', 'Kum', 'Kee']
['2', 'łyżki', 'stołowe', 'sosu', 'sojowego', 'słodkiego', 'do', 'pierożków', 'Lee', 'Kum', 'Kee']
['50', 'g', 'kasztanów', 'jadalnych', 'z', 'puszki', 'w', 'wodzie,', 'pokrojonych', 'w', 'plastry', '(opcjonalnie)']
[]
['•', '2', 'łyżki', 'stołowe', 'sosu', 'do', 'pierożków', 'Lee', 'Kum', 'Kee']
['120g', 'makaronu', 'Mie', 'lub', 'z', 'warzywami']
['200g', 'filetu', 'z', 'kurczaka']
['1', 'łyżka', 'mąki', 'p

['2', 'łyżki', 'sezamu']
['2', 'łyżki', 'pasty', 'miso']
['2', 'łyżki', 'sosu', 'sojowego']
['1', 'limonka']
['1/2', 'łyżeczki', 'cukru']
['1', 'łyżka', 'oleju', 'sezamowego']
['4', 'łyżki', 'posiekanego', 'szczypiorku']
['świeża', 'kolendra']
['800g', 'filetu', 'z', 'łososia']
['1', 'łyżka', 'oliwy']
['Marynata:']
['6', 'łyżek', 'sosu', 'sojowego']
['8', 'łyżek', 'sosu', 'teriyaki']
['2', 'łyżki', 'miodu', 'lipowego']
['1', 'czubata', 'łyżeczka', 'imbiru', '(mielonego', 'lub', 'świeżo', 'startego', '-', '1,5cm)']
['2', 'duże', 'żabki', 'czosnku', '(przeciśnięte', 'przez', 'praskę', 'i', 'roztarte', 'z', 'solą)']
['sól', 'i', 'pieprz']
['Tempura', 'Samlip']
['olej', 'kokosowy']
['surowe', 'krewetki', '24', 'szt.']
['mąka', 'pszenna', '1', 'szklanka']
['sól']
['woda']
['Bulion:', '•', '1kg', 'kości', 'wieprzowych']
['1kg', 'kości', 'drobiowych']
['4', 'cebulki', 'szalotki']
['10cm', 'korzenia', 'imbiru']
['pół', 'główki', 'czosnku']
['2', 'marchewki']
['albi', 'Konbu', '(płat)']
['100ml

['2', 'łyżki', 'stołowe', 'oleju', 'roślinnego']
['1', 'łyżka', 'stołowa', 'sosu', 'rybnego']
['1', 'filiżanka', 'mleczka', 'kokosowego']
['1/4', 'filiżanki', 'liści', 'słodkiej', 'bazylii']
['8', 'liści', 'limonki', 'kaffir,', 'drobno', 'posiekanych']
['1/3', 'łyżki', 'stołowej', 'jasnego', 'sosu', 'sojowego']
['2', 'łyżki', 'stołowe', 'pasty', 'curry', 'panang']
['15', 'ml', 'oleju', 'do', 'woka', 'Thai', 'Pride']
['40', 'g', 'żółtej', 'pasty', 'curry', 'Thai', 'Pride']
['40', 'ml', 'wody']
['60', 'g', 'ziemniaków']
['150', 'g', 'mięsa', 'drobiowego']
['60', 'g', 'cebuli']
['160', 'ml', 'mleczka', 'kokosowego', 'Diamond']
['20', 'g', 'liści', 'bazylii']
['5', 'g', 'czerwonej', 'papryczki', 'chili']
['1', 'pierś', 'z', 'kurczaa']
['3', 'liście', 'limonki', 'kafir']
['1-2', 'łyżki', 'pasty', 'Panang']
['1', 'łyżeczka', 'sosu', 'rybnego']
['1', 'łyżeczka', 'cukru', 'trzcinowego', 'lub', 'palmowego']
['1', 'łyżka', 'oleju']
['100', 'ml', 'mleczka', 'kokosowego']
['2', 'liście', 'limonki'

['30', 'ml', 'soku', 'z', 'limonki']
['1/2', 'szklanki', 'nasion', 'granatu']
['2', 'szt.', 'zielonej', 'części', 'dymki']
['sól']
['pieprz']
['listki', 'mięty', 'do', 'dekoracji']
['jajka', '5', 'sztuk']
['soczewica', '100', 'g']
['cebula', 'czerwona', '50', 'g']
['papryczka', 'jalapeno', 'marynowana', '50', 'g']
['majonez', '1⁄2', 'szklanki']
['musztarda', 'Dijon', '10', 'g']
['sól']
['pieprz']
['natka', 'pietruszki']
['500', 'g', 'ciecierzycy']
['1', 'czerwona', 'papryka']
['4', 'łyżki', 'oliwy', 'z', 'oliwek']
['1', 'łyżka', 'soku', 'z', 'cytryny']
['1/2', 'łyżeczki', 'cukru']
['3', 'łyżeczki', 'kminu', 'rzymskiego']
['1', 'łyżeczka', 'płatków', 'chili']
['mała', 'szczypta', 'soli']
['1/2', 'łyżeczki', 'mielonego', 'pieprzu']
['1', 'ząbek', 'czosnku']
['1/3', 'młodej', 'cebuli']
['1', 'pomidor']
['3', 'łyżki', 'posiekanego', 'koperku']
['4', 'łyżki', 'posiekanej', 'natki', 'pietruszki']
['2', 'łyżki', 'posiekanego', 'szczypiorku']
['kilka', 'listków', 'mięty']
['2', 'łyżeczki', 'ol

['1', 'szklanka', 'fasoli', 'białej', '(fasolę', 'wcześniej', 'namoczyć)']
['1/2', 'pora']
['1', 'łyżeczkę', 'mielonego', 'kminu']
['skórka', 'z', 'połowy', 'cytryny']
['olej']
['4', 'laski', 'selera']
['2', 'ziemniaki']
['2', 'łyżki', 'pasty', 'Harissa']
['2', 'liście', 'laurowe']
['1,5-2', 'l', 'bulionu']
['sok', 'z', 'połowy', 'cytryny']
['pęczek', 'kolendry']
['sól']
['pieprz', 'czarny']
['2', 'cukinie']
['1', 'cebula']
['140', 'g', 'pomidorów', 'suszonych', 'w', 'zalewie']
['2', 'ząbki', 'czosnku']
['150', 'g', 'świeżego', 'szpinaku']
['200', 'g', 'koziego', 'sera', '(rolada)']
['450', 'g', 'pomidorów', 'z', 'puszki']
['szczypta', 'cynamonu']
['1', 'łyżeczka', 'suszonego', 'oregano']
['1/2', 'łyżeczki', 'wędzonej', 'słodkiej', 'papryki']
['brązowy', 'cukier']
['miód']
['syrop', 'z', 'agawy', '(opcjonalnie)']
['2', 'łyżki', 'harissy']
['sól']
['pieprz']
['oliwa', 'z', 'oliwek']
['250g', 'mielonej', 'wieprzowiny']
['¼', 'łyżeczki', 'sody', 'oczyszczonej']
['2', 'łyżki', 'pasty', 'ha

['1', 'łyżka', 'jasnego', 'sosu', 'sojowego']
['½', 'łyżki', 'sosu', 'rybnego']
['1', '½', 'łyżki', 'cukru']
['2', 'łyżki', 'prażonych', 'nasion', 'białego', 'sezamu']
['1', 'szklanka', 'ciepłego', 'mleka']
['2', 'szklanki', 'mąki']
['2', 'łyżki', 'cukru']
['2', 'łyżeczki', 'suszonych', 'drożdży']
['szczypta', 'soli']
['1', 'łyżka', 'oleju']
['brązowy', 'cukier']
['parę', 'łyżek', 'posiekanych', 'orzeszków', '(włoskich,', 'laskowych', 'lub', 'ziemnych)']
['1', 'łyżeczka', 'cynamonu']
['1', 'opakowanie', 'zupki', 'Chapaghetti']
['1', 'opakowanie', 'zupki', 'Neoguri', '(ostra', '/', 'łagodna)']
['6', 'szklanek', 'wody', '(ok.', '1,1l)']
['340', 'g', 'tofu']
['200', 'g', 'gorzkiej', 'czekolady']
['1/2', 'szklanki', 'wody']
['1/2', 'szklanki', 'cukru']
['dodatki', 'smakowe', 'np.', '1', 'łyżeczka', 'ekstraktu', 'z', 'wanilii,', '1', 'łyżka', 'likieru', 'pomarańczowego,', '1/3', 'łyżeczki', 'cynamonu,', 'szczypta', 'ostrej', 'papryki']
['45', 'ml', 'Mekhong', 'Thai', 'Spirit']
['25', 'ml', 

In [88]:
jednostki = [
    'łyżeczki',
    'łyżki',
    'szklanki',
    'szczypta',
    'ząbek',
    'garść',
    'l',
    'ml',
    'g',
    'kawałki',
    'puszka',
    'opakowanie',
    'karton',
    'filiżanka',
    'gałązka',
    'cząstka',
    'liści',
    'plastry',
    'sztuk'
]
nie_jednostki = [
    'jogurtu',
    'owoców',
    'banany',
    'ostrej',
    'czerwonej',
    'piersi',
    'szparagów',
    'kapustka',
    'cukier',
    'świeżego',
    'dojrzałe',
    'Rumu',
    'dojrzałe',
    'czystej',
    'smakowe',
    'tequili',
    'limonki'
]
jednostki = Sentence(" ".join(jednostki))
nie_jednostki = Sentence(" ".join(nie_jednostki))
emb.embed(jednostki)
emb.embed(nie_jednostki)

[Sentence: "jogurtu owoców banany ostrej czerwonej piersi szparagów kapustka cukier świeżego dojrzałe Rumu dojrzałe czystej smakowe tequili limonki" - 17 Tokens]

In [90]:
X_jed = torch.cat([j.embedding for j in jednostki]).reshape(19,300)
X_njed = torch.cat([j.embedding for j in nie_jednostki]).reshape(17,300)

In [123]:
def knn(Y):
    mask = Y.norm(dim=1) != 0

    cos_sim_jed = X_jed @ Y[mask,:].T / (X_jed.norm(dim=1, keepdim=True) * Y[mask,:].norm(dim=1, keepdim=True).T)
    cos_sim_njed = X_njed @ Y[mask,:].T / (X_njed.norm(dim=1, keepdim=True) * Y[mask,:].norm(dim=1, keepdim=True).T)

    ret = torch.zeros(Y.shape[0], dtype=torch.bool).to(torch.device('cuda'))

    ret[mask] = (cos_sim_jed.max(0)[0] > cos_sim_njed.max(0)[0]) & (cos_sim_jed.max(0)[0] > 0.5)
    return ret

In [124]:
def sentenceToMat(s):
    s = Sentence(s)
    emb.embed(s)
    r = torch.cat([i.embedding for i in s])
    return r.reshape(int(r.shape[0]/300), 300)

In [130]:
knn(sentenceToMat('1 łyżka bezalkoholowy z guawą i imbirem mega'))

tensor([False,  True, False, False, False, False, False, False],
       device='cuda:0')

In [118]:
for d in data:
    for i in d['ingredients']:
        knn(sentence)


tensor([[ 0.2346,  0.2216,  0.0498,  ...,  0.0324, -0.0897,  0.1845],
        [-0.1335,  0.2546,  0.0863,  ...,  0.0548, -0.0453,  0.2652],
        [-0.3744, -0.0344, -0.4256,  ...,  0.0160,  0.0595,  0.1967],
        [-0.1382, -0.0861, -0.0652,  ..., -0.0801, -0.2138, -0.2443],
        [-0.4339,  0.4223, -0.4476,  ..., -0.1537,  0.2591,  0.2551],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       device='cuda:0')
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0938,  0.1546,  0.1199,  ...,  0.0672, -0.1122,  0.0197],
        [-0.2510, -0.3749, -0.4399,  ..., -0.1340,  0.1896,  0.3422],
        ...,
        [ 0.0420,  0.3535,  0.0200,  ..., -0.1521,  0.1627,  0.0068],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       device='cuda:0')
tensor([[ 0.2346,  0.2216,  0.0498,  ...,  0.0324, -0.0897,  0.1845],
        [-0.2585, -0.0234, 

       device='cuda:0')
